In [14]:
from new_methods import *

In [37]:
process_dict = {'CO': ['08093', '08119'],
                'FL': ['12086', '12011', '12087', '12051', '12099'],
                'LA': ['22071', '22051', '22075', '22087', '22103', '22057', '22089', '22095', '22105']}

for co, ls in process_dict.items():
    for l in ls:
        AE = False
        s = datetime.now()
        for fi in glob.glob(f'{co}_FZ\*{l}.shp'):
            selFZ = gpd.read_file(os.path.abspath(fi))
            print('Processing', fi)
            
        for fi in glob.glob(f'{co}_BFEs\*{l}.shp'):
            selBFE = gpd.read_file(os.path.abspath(fi))
            print('Processing', fi)
        # determine correct UTM projection from flood zone
        crs = utm_code(selFZ)

        print('AE Zones')
        # Locate all AE and A* zones with no Static values
        sel_ae = selFZ.loc[(selFZ['ZONE'].str.contains('A.')) & 
        (selFZ['ZONE'] != 'A99') & 
        (selFZ['ZONE'] != 'AO') & 
        (selFZ['ZONE'] != 'AR')
        ]

        sel_ae_dynamic = sel_ae.loc[sel_ae['BFE_STATIC'] == -9999.0]
        sel_ae_dis = sel_ae_dynamic.dissolve(by='FIPS')
        sel_ae_dis.to_crs(crs, inplace=True)
        ae_zones = sel_ae_dis[['geometry']]
        ae_zones.reset_index(inplace=True)

        print('A Zones')
        sel_a = selFZ.loc[selFZ['ZONE'] == 'A']

        print('X500 Zones')
        # Locate x500 and prep
        sel_x = selFZ.loc[selFZ['ZONE'] == 'X500']
        if len(sel_x) > 0:
            x = x.dissolve().explode().to_crs(crs)[['geometry']]
            x = x.reset_index(0, drop=True)
            x.to_file(f'{l}_x500.shp')

        
        # If there are no AE or A* Zones (i.e. dataframe is empty, continue on to the next county)
        if len(ae_zones) > 0:
            AE = True
            print('Cleaning BFEs')
            # Reproject BFEs 
            selBFE.to_crs(crs, inplace=True)
            sel_bfe = selBFE[['ELEV', 'geometry']]
            sel_bfe = sel_bfe.drop_duplicates('geometry') # * delete late

            """ If main Index joins with other Indices, this means that a BFE is broken
             up into several linestrings. This cleaning process attempts to merge
            all linestrings into one per BFE """

            bfe_bfe = sel_bfe.sjoin(sel_bfe, how='left')
            bfe_bfe.reset_index(inplace=True)

            brok = bfe_bfe.loc[bfe_bfe['index'] != bfe_bfe['index_right']]
            sel_bfe.reset_index(inplace=True)

            if len(brok) > 0:
                # df with both main geometry and adjacent geoms
                fix = brok.merge(sel_bfe, left_on='index_right', right_on='index')

                """Locating and gathering adjacent geometries to linestrings in the middle. These will account for 
                BFEs that have been broken up more than twice """
                # Series with indice count of spatial join
                vals = fix['index_x'].value_counts()

                # Obtaining the Indices that touch more than one linestring
                mid = fix.loc[fix['index_x'].isin(vals.index[vals.gt(1)])]

                # Groupby Main Index and collect adj. geoms into list
                mid_gb = mid.groupby('index_x')['geometry_y'].apply(list)

                # Groupby Main Index and collect adj. indices into list
                mid_adj = mid.groupby('index_x')['index_right'].apply(list)

                # Build df from groupby series to join and obtain key geometry
                mid_df = pd.DataFrame(mid_gb)
                mid_df.reset_index(inplace=True)
                mid_join = mid_df.merge(sel_bfe, left_on='index_x', right_index=True)

                """ Adjacent IDS to Middle that need to be removed since these will be apart of the 3 linestring merge """
                # Creating list of Main Indices that touch more than one linestring
                mid_ids = mid['index_x'].drop_duplicates().to_list()

                # Creating list of adjacent indices that touch Main Index
                adj_list = [ids for m in mid_adj.to_list() for ids in m]
                adj_list = adj_list + mid_ids  

                """ These should be 2 linestrings ONLY """
                # Remove ids from adj_list, leaving ids that touch only One other linestring
                fix_rem = fix.loc[~fix['index_x'].isin(adj_list)]

                """ Correct 2 linestring breaks """
                fix_rem['new_geom'] = fix_rem.apply(lambda x: linemerge(list((x.geometry_x, x.geometry_y))), axis=1 )
                fix_df = fix_rem[['index_x', 'ELEV_left', 'new_geom']].rename(columns={'index_x': 'index', 'new_geom': 'geometry', 'ELEV_left': 'ELEV'}).set_geometry('geometry', crs=crs)

                # This removes the other pair in the 1-to-1 join
                fix_df = fix_df.drop_duplicates('geometry')

                """ Correct 3 (or more) linestring breaks """
                if len(mid_join) > 0:
                    mid_join['new_geom'] = mid_join.apply(lambda x: linemerge(x['geometry_y'] + [x['geometry']]), axis=1)
                    mid_join = mid_join[['index_x', 'ELEV', 'new_geom']].rename(columns={'index_x': 'index', 'new_geom': 'geometry'}).set_geometry('geometry', crs=crs)

                    bfe_cleaned = pd.concat([mid_join, fix_df], names=['ELEV', 'geometry'])
                    bfe_c_geom = bfe_cleaned[['geometry']]
                   
                    # spatial join cleaned geometry to original bfe df
                    clean_join = sel_bfe.sjoin(bfe_c_geom, how='left')
                    
                    # Locate bfes that were not cleaned
                    sel_bfe = clean_join.loc[clean_join['index_right'].isnull(), ['ELEV', 'geometry']]

                    # Final Cleaned BFE dataframe
                    bfe = pd.concat([bfe_cleaned, sel_bfe], ignore_index=True)
                else:
                    fix_df = fix_df[['ELEV', 'geometry']]
                    bfe_c_geom = fix_df[['geometry']]
                    clean_join = sel_bfe.sjoin(bfe_c_geom, how='left')
                    sel_bfe = clean_join.loc[clean_join['index_right'].isnull(), ['ELEV', 'geometry']]
                
                    bfe = pd.concat([fix_df, sel_bfe], ignore_index=True)
                
                
            else:
                bfe = sel_bfe[['ELEV', 'geometry']]

            # final removal of any multiline artifacts
            bfe = remove_multiline_BFE(bfe)
            
            # If conversion occured, there would by multiple linestrings that now have duplicate geometries. Need removal
            bfe = bfe.drop_duplicates('geometry')

            # repair geometries before following phase
            bfe['rgeom'] = bfe.apply(lambda x: make_valid(x.geometry), axis=1)
            bfe = bfe[['ELEV', 'rgeom']].rename(columns={'rgeom': 'geometry'}).set_geometry('geometry', crs=crs)

            # Extend BFEs over FSP Poly then reset bfe to new position
            print('Extending BFE...')
            bfe_ext, short_bfe, corrupt = bfe_extend(bfe, ae_zones, crs=crs)

            if len(corrupt) > 0:
                print(l, 'Corrupt BFE')
                print(corrupt)


            print('Splitting FSP...')
            # Split FSP Poly by extended BFEs
            ae_s = split_fsp(ae_zones, bfe_ext, crs=crs)

            print('Cleaning FSP...')
            # Remove slivers due to overlaps from extended BFEs
            ae_s = ae_s.overlay(ae_zones)
            ae_s = ae_s.loc[ae_s.geom_type == 'Polygon']
            ae_s = ae_s.loc[ae_s.area > 1] # Removing small artifacts from cleaning and split process
            ae_s = ae_s[['geometry']]
            ae_s.reset_index(inplace=True)

            ae_s_84 = ae_s.to_crs(4326)
            #ae_s_84.to_file(f'{l}_FloodZone_split.shp')

            # add short bfes to final bfe dataframe
            bfe = pd.concat([bfe_ext, short_bfe])
            bfe_84 = bfe.to_crs(4326)
            bfe_84.to_file(f'{l}_BFE.shp')

        else:
            print('No AE Zones')
            pass

        """ A Zone Merge. AE Zones with only 1 BFE will include End portions of the 
        Flood Zone and have the potential to be connect to AZones. This transition
        in Zone type is what needs to be located, so that the AE Zones that touch 
        an Azone can be converted to an AZone. Once all Azones are then accounted for
        these will be ready for triangulation. """
        if len(sel_a) > 0:
            print('AZones Present')
            a_zones = sel_a.dissolve().explode().reset_index(0, drop=True).to_crs(crs)
            
            # Identiify AE Zones and their assoicated BFEs. 
            ae_join = ae_s.sjoin(bfe)
            vals = ae_join.index.value_counts()

            # Find polygons with single BFE
            ae_single = ae_join.loc[ae_join.index.isin(vals.index[vals.lt(2)])]
            ae_single = ae_single[['geometry']]
            
            # join azones to ae zones (inner)
            ajoin = a_zones.sjoin(ae_single)[['geometry', 'index_right']]

            # Condition if any azones touch ae zones
            if len(ajoin) > 0:
                print('Joining AE Zones to A Zones')
                # ae indices to drop from original ae dataframe
                ae_drops = ajoin['index_right'].to_list()
                ae_s = ae_s.loc[~ae_s.index.isin(ae_drops)]

                # obtain geometries from matched indicies
                ajoin_geoms = ajoin.merge(ae_single, left_on='index_right', right_index=True, how='left')

                # groupby A-Zone geometry and agg intersecting ae geometry
                a_gb = ajoin_geoms.groupby(level=0)['geometry_y'].apply(list)
                a_merge = a_zones.merge(a_gb, left_index=True, right_index=True)

                # union polygons
                a_merge['new_geom'] = a_merge.apply(lambda x: unary_union([x.geometry] + x.geometry_y), axis=1)

                # create new gdf from new geometry
                a_merge = a_merge[['new_geom']].rename(columns={'new_geom': 'geometry'}).set_geometry('geometry', crs=crs)
                
                # Concat to orginal Azone dataframe
                a_zones = a_zones.loc[~a_zones.index.isin(a_merge.index)]
                all_a_zones = pd.concat([a_zones, a_merge])
                
                # Final FZ dataset
                ae_s['fz_type'] = 'AE'
                all_a_zones['fz_type'] = 'A'
                all_a_ae = pd.concat([all_a_zones, ae_s])
                all_a_ae_84 = all_a_ae[['fz_type', 'geometry']].to_crs(4326)
                
                all_a_ae_84.to_file(f'{l}_FZ.shp')
                print(f'Processing time for County {l}:', (datetime.now() - s))
                print('______________________________________________________')
                
                

            else:
                a_zones['fz_type'] = 'A'
                a_zones = a_zones[['fz_type', 'geometry']].to_crs(4326)
                a_zones.to_file(f'{l}_FZ.shp')
                print(f'Processing time for County {l}:', (datetime.now() - s))
                print('______________________________________________________')
        
        else:
            print('No A Zones')
            print(f'Processing time for County {l}:', (datetime.now() - s))
            print('______________________________________________________')
            continue

        
                    


                    

Processing CO_FZ\FM_08093.shp
AE Zones
A Zones
X500 Zones
No AE Zones
AZones Present
Processing time for County 08093: 0:00:01.848619
______________________________________________________
Processing CO_FZ\FM_08119.shp
Processing CO_BFEs\EL_08119.shp
AE Zones
A Zones
X500 Zones
Cleaning BFEs
Extending BFE...
polygon to line 0:00:00.001949
ends 0:00:00.104444
parse end points 0:00:00.355457
create start point for extapolation 0:00:00.427717
create extrapolation 0:00:00.586933
intersection 0:00:00.881828
remove empty linestrings 0:00:00.010732
get nearest point 0:00:00.715839
create ext_line 0:00:00.809093
combine extension 0:00:00.903279
new ends 0:00:00.102561
parse new ends 0:00:00.338888
buffer 0:00:00.396473
symmetric difference 0:00:01.104463
create rep point 0:00:00.463894
final ext line 0:00:00.569319
create final line 0:00:00.453162
final clean up 0:00:00.018543
Splitting FSP...
Cleaning FSP...
AZones Present
Processing time for County 08119: 0:00:14.008833
_____________________

In [50]:
df = pd.DataFrame({'ELEV': [10, 10, 11, 15]})
df1 = pd.DataFrame({'ELEV': [10, 10, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 15, 10, 10, 10, 10, 10, 10, 10]})

In [51]:
df.describe()

,ELEV
count,4.000000
mean,11.500000
std,2.380476
min,10.000000
25%,10.000000
50%,10.500000
75%,12.000000
max,15.000000


In [52]:
df1.describe()

,ELEV
count,20.000000
mean,10.300000
std,1.128576
min,10.000000
25%,10.000000
50%,10.000000
75%,10.000000
max,15.000000
